# SQL Evaluator Agent Test

This notebook tests the SQLEvaluatorAgent which automatically:
1. Finds SQL from the query tree node
2. Executes the SQL against the database
3. Analyzes the results to determine if they answer the user's intent

## Key Requirements
- The database path must be set in memory with key "database_path"
- The node must have SQL generated (status should be 'sql_generated')
- The agent is invoked with format: `node:{node_id} - Analyze SQL execution results`

In [1]:
import os
import sys
import asyncio
import sqlite3
import json
import logging
import re
from typing import Dict, Any, List, Optional
from dotenv import load_dotenv

sys.path.append('../src')
load_dotenv()

# Set up logging
logging.basicConfig(level=logging.INFO, 
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# Reduce noise from autogen
logging.getLogger('autogen_core').setLevel(logging.WARNING)

In [2]:
from pathlib import Path
from keyvalue_memory import KeyValueMemory
from task_context_manager import TaskContextManager
from query_tree_manager import QueryTreeManager
from database_schema_manager import DatabaseSchemaManager
from node_history_manager import NodeHistoryManager
from query_analyzer_agent import QueryAnalyzerAgent
from schema_reader import SchemaReader
from memory_content_types import (
    TaskContext, QueryNode, NodeStatus, TaskStatus,
    QueryMapping, TableMapping, ColumnMapping, JoinMapping,
    TableSchema, ColumnInfo, CombineStrategyType
)
from sql_evaluator_agent import SQLEvaluatorAgent

data_path = "/home/norman/work/text-to-sql/MAC-SQL/data/bird"
tables_json_path = Path(data_path) / "dev_tables.json"
db_name = "california_schools"

In [3]:
task_id = "experimental-test"

query = "What is the highest eligible free rate in Alameda County?"
intent="Find the maximum eligible free rate for K-12 students in schools located in Alameda County"
memory = KeyValueMemory()

# Initialize task
task_manager = TaskContextManager(memory)
await task_manager.initialize(task_id, query, db_name)

# Load schema - this will automatically store data_path and dataset_name in metadata
schema_manager = DatabaseSchemaManager(memory)
# Note: we no longer call initialize() here since load_from_schema_reader will do it

schema_reader = SchemaReader(
    data_path=data_path,
    tables_json_path=str(tables_json_path),
    dataset_name="bird",
    lazy=False
)
await schema_manager.load_from_schema_reader(schema_reader, db_name)

sql = """
    SELECT MAX(f."Eligible Free Rate (K-12)") as max_rate
    FROM schools s
    JOIN frpm f ON s.CDSCode = f.CDSCode
    WHERE s.County = 'Alameda'
    """
tree_manager = QueryTreeManager(memory)
        
# Create node
node_id = await tree_manager.initialize(intent)

# Update node with SQL
await tree_manager.update_node_sql(node_id, sql)

2025-05-25 08:41:44,237 - TaskContextManager - INFO - Initialized task context for task experimental-test


load json file from /home/norman/work/text-to-sql/MAC-SQL/data/bird/dev_tables.json

Loading all database info...
Found 11 databases in bird dataset


2025-05-25 08:41:56,680 - DatabaseSchemaManager - INFO - Initialized empty database schema
2025-05-25 08:41:56,680 - DatabaseSchemaManager - INFO - Added table 'frpm' to schema
2025-05-25 08:41:56,681 - DatabaseSchemaManager - INFO - Added table 'satscores' to schema
2025-05-25 08:41:56,682 - DatabaseSchemaManager - INFO - Added table 'schools' to schema
2025-05-25 08:41:56,682 - DatabaseSchemaManager - INFO - Loaded schema for database 'california_schools' with 3 tables
2025-05-25 08:41:56,682 - QueryTreeManager - INFO - Initialized query tree with root node node_1748176916.682731_root
2025-05-25 08:41:56,683 - QueryTreeManager - INFO - Updated node node_1748176916.682731_root


In [4]:
agent = SQLEvaluatorAgent(memory, llm_config={
    "model_name": "gpt-4o",
    "temperature": 0.1,
    "timeout": 60
}, debug=True)

2025-05-25 08:41:56,703 - SQLEvaluatorAgent - DEBUG - Created AssistantAgent: sql_evaluator
2025-05-25 08:41:56,703 - SQLEvaluatorAgent - DEBUG - Created MemoryAgentTool for sql_evaluator
2025-05-25 08:41:56,704 - SQLEvaluatorAgent - INFO - Initialized sql_evaluator with model gpt-4o


In [5]:
content = await memory.show_all('summary')
print(content)

=== Memory Store Summary ===
Total items: 7

Keys (most recent values):
  - queryTree (JSON)
  - databaseSchema (JSON)
  - taskContext (JSON)


In [6]:
# Verify the node has SQL
node = await tree_manager.get_node(node_id)
print(f"Node ID: {node_id}")
print(f"Node Intent: {node.intent}")
print(f"Node SQL: {node.sql}")
print(f"Node Status: {node.status}")
print(f"Has execution result: {node.executionResult is not None}")

Node ID: node_1748176916.682731_root
Node Intent: Find the maximum eligible free rate for K-12 students in schools located in Alameda County
Node SQL: 
    SELECT MAX(f."Eligible Free Rate (K-12)") as max_rate
    FROM schools s
    JOIN frpm f ON s.CDSCode = f.CDSCode
    WHERE s.County = 'Alameda'
    
Node Status: NodeStatus.SQL_GENERATED
Has execution result: False


In [7]:
# Check the database schema metadata
db_schema = await memory.get("databaseSchema")
if db_schema and "metadata" in db_schema:
    print("Database Schema Metadata:")
    for key, value in db_schema["metadata"].items():
        print(f"  {key}: {value}")
else:
    print("No metadata found in database schema")

Database Schema Metadata:
  data_path: /home/norman/work/text-to-sql/MAC-SQL/data/bird
  dataset_name: bird
  database_id: california_schools


In [8]:
# Now run the SQL evaluator with the node_id
# The agent should automatically find the SQL from the node and execute it
result = await agent.run(f"node:{node_id} - Analyze SQL execution results")

2025-05-25 08:41:56,724 - SQLEvaluatorAgent - INFO - Executing SQL for node node_1748176916.682731_root on database california_schools
2025-05-25 08:41:56,724 - SQLEvaluatorAgent - DEBUG - Using data_path: /home/norman/work/text-to-sql/MAC-SQL/data/bird, dataset_name: bird
2025-05-25 08:41:56,724 - QueryTreeManager - INFO - Updated node node_1748176916.682731_root
2025-05-25 08:41:56,725 - SQLEvaluatorAgent - DEBUG - SQL evaluator context prepared with result status: success


[SQLExecutor] Connecting to database: /home/norman/work/text-to-sql/MAC-SQL/data/bird/dev_databases/california_schools/california_schools.sqlite


2025-05-25 08:42:00,823 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-25 08:42:00,826 - SQLEvaluatorAgent - INFO - Stored analysis for node node_1748176916.682731_root - Answers intent: no, Quality: poor
2025-05-25 08:42:00,826 - SQLEvaluatorAgent - INFO - Analysis complete - Answers intent: no, Quality: poor


In [9]:
# Get the analysis results
analysis = await memory.get("execution_analysis")
if analysis:
    print("Analysis Results:")
    print(f"  Answers Intent: {analysis.get('answers_intent')}")
    print(f"  Result Quality: {analysis.get('result_quality')}")
    print(f"  Result Summary: {analysis.get('result_summary')}")
    print(f"  Confidence Score: {analysis.get('confidence_score')}")
    
    if analysis.get('issues'):
        print("\nIssues Found:")
        for issue in analysis['issues']:
            print(f"  - Type: {issue.get('type')}, Severity: {issue.get('severity')}")
            print(f"    Description: {issue.get('description')}")
    
    if analysis.get('suggestions'):
        print("\nSuggestions:")
        for suggestion in analysis['suggestions']:
            print(f"  - {suggestion}")
else:
    print("No analysis results found")

Analysis Results:
  Answers Intent: no
  Result Quality: poor
  Result Summary: The query execution returned no data, indicating that it did not successfully find the maximum eligible free rate for K-12 students in Alameda County.
  Confidence Score: 0.7

Issues Found:
  - Type: data_quality, Severity: high
    Description: The result set is empty, which suggests that there may be no matching records in the database or an issue with the data.
  - Type: logic, Severity: high
    Description: The query may not be correctly identifying or joining the relevant tables, or there may be no data for Alameda County in the 'schools' or 'frpm' tables.

Suggestions:
  - Verify that the 'schools' and 'frpm' tables contain data for Alameda County and that the 'CDSCode' fields are correctly populated and matched.
  - Check if there are any filters or conditions in the database that might be excluding Alameda County data.
  - Consider adding error handling or logging to capture more information about 

In [10]:
# Check the messages
for message in result.messages:
    print(f"\n[{getattr(message, 'source', 'Unknown')}]:")
    print(message.content)
    print("-" * 80)


[user]:
I'm providing you with context from previous interactions:

### Node Id
node_1748176916.682731_root

### Intent
Find the maximum eligible free rate for K-12 students in schools located in Alameda County

### Sql

    SELECT MAX(f."Eligible Free Rate (K-12)") as max_rate
    FROM schools s
    JOIN frpm f ON s.CDSCode = f.CDSCode
    WHERE s.County = 'Alameda'
    

### Execution Result
- status: success
- columns: []
- data: []
- row_count: 0
- execution_time: None

node:node_1748176916.682731_root - Analyze SQL execution results
--------------------------------------------------------------------------------

[sql_evaluator]:
<evaluation>
  <answers_intent>no</answers_intent>
  <result_quality>poor</result_quality>
  <result_summary>The query execution returned no data, indicating that it did not successfully find the maximum eligible free rate for K-12 students in Alameda County.</result_summary>
  <issues>
    <issue>
      <type>data_quality</type>
      <description>The r